In [1]:
import sys
print(sys.prefix)
import os
os.listdir('data')

/home/debboutr/anaconda3/envs/gis


['HermanCreek_Hexs_SMALLER.csv',
 'HermanCreekHexs.dbf',
 'HermanCreekHexs.shp',
 'HermanCreekHexs.sbx',
 'HermanCreekHexs.shx',
 'thermal_patch_temp.csv',
 'hexes.json',
 'HermanCreekHexs.xml',
 'HermanCreekHexs.cpg',
 'HermanCreekHexs.prj',
 'hexy.qgs',
 'smallerTEST.log',
 'HEX-A-GONE-FISHIN-20190426T191258Z-001.zip',
 'HermanCreekHexs.sbn']

In [2]:
from IPython.display import display
import geopandas as gpd
hexes = gpd.read_file('data/HermanCreekHexs.shp')
hexes.head()

,Join_Count,TARGET_FID,Hex_ID,Score,refuges_fi,sparse_dep,Shape_Leng,Hex_ID2,Refuge_ID,depth_m,ORIG_FID,Shape_Le_1,Shape_Area,geometry
0,1,0,87328693.0,0.0,2.0,1.902400,83.235790,1295096,2,1.902400,197,83.235790,499.999532,"POLYGON ((-1990921.9562 2794589.769300001, -19..."
1,1,1,87360579.0,0.0,2.0,0.584366,83.235690,1315484,2,0.584366,280,83.235690,499.998330,"POLYGON ((-1990945.984300001 2794548.1514, -19..."
2,1,2,87360575.0,0.0,2.0,1.084370,83.235863,1315480,2,1.084370,276,83.235863,500.000411,"POLYGON ((-1991042.096799999 2794548.1514, -19..."
3,1,3,87376517.0,0.0,2.0,0.709759,83.235890,1325673,2,0.709759,312,83.235890,500.000733,POLYGON ((-1991078.138900001 2794527.342499999...
4,1,4,87488139.0,0.0,2.0,2.821340,83.235890,1397052,2,2.821340,514,83.235890,500.000733,"POLYGON ((-1990657.6469 2794381.6798, -1990645..."


In [3]:
hexes.crs

{'init': 'epsg:5070'}

In [4]:
print(hexes.crs)
hexes.to_crs({'init':'epsg:4326'}, inplace=True)
print(hexes.crs)
hexes.to_file('data/hexes.json', driver='GeoJSON')

{'init': 'epsg:5070'}
{'init': 'epsg:4326'}


In [5]:
extent = hexes.unary_union.bounds
lat_Center = (extent[1] + extent[3]) / 2
lon_Center = (extent[0] + extent[2]) / 2
print(lat_Center,lon_Center)

45.684395820853425 -121.8618638568064


In [7]:
# import pandas as pd
# heat = pd.read_csv('data/hexsim/hex_temp.csv')
# heat.drop(['CWR_ID','SIZE_RANK'],axis=1,inplace=True)
# heat.head()

In [ ]:
# dep = pd.read_csv('data/hexsim/reach1_500_depth_m.csv')
# len(dep)

In [8]:
# hexes = hexes.merge(dep, on='Hex_ID')
# hexes = hexes.merge(heat, on='Hex_ID')

In [6]:
import folium
from folium import plugins

hex_map = folium.Map(location=[lat_Center,lon_Center], 
                               tiles="Stamen Terrain", 
                               zoom_start=15)

geojson = folium.GeoJson('./data/hexes.json', 
                            name='Hexes')

hex_map.add_child(geojson)
folium.LayerControl().add_to(hex_map)
plugins.FloatImage('https://debboutr.github.io/epa_clipped.png', bottom=74, left=8).add_to(hex_map)

hex_map

In [ ]:
m = folium.Map(
    location=[lat_Center,lon_Center],
    tiles='Stamen Terrain',
    zoom_start=15)

m.choropleth(
    geo_data='./data/hexes.json',
    data=pd.read_csv('./data/hexsim/reach1_500_depth_m.csv'),
    columns=['Hex_ID','Depth_m'],
    key_on='feature.properties.Hex_ID',
    name='Hexes',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
#     line_color='white',
#     highlight=True,
    legend_name='Depth (m)')

folium.LayerControl().add_to(m)
# m.save('./maps/hex_depth.html')
m

In [ ]:
(45.684395820853425, -121.8618638568064)

In [50]:
lat_Center= 45.684395820853425
lon_Center= -121.865

In [51]:
hex_map = folium.Map(location=[lat_Center,lon_Center], 
                               tiles=("https://api.mapbox.com/styles/v1/mapbox/satellite-v9/tiles/256/{z}/{x}/{y}?"
                                      "access_token=pk.eyJ1IjoiZGFyYW9iZWlybmUiLCJhIjoidlV2WXZKRSJ9."
                                      "EwVOY6b8UeyIz3N2UAu5pw"), 
                               zoom_start=16, attr='@Mapbox')

json = folium.GeoJson('./data/hexes.json', 
                      name='Hexes',
                      style_function=lambda x: {     'fillColor' :'red', 
                                                  'fill_opacity' : 0.47, 
                                                         'color' : 'green',
                                                        'weight' : 2},
                      highlight_function= lambda x: {'fillColor' : 'blue',
#                                                          'color' : 'yellow',
                                                        'weight' : 3})

hex_map.add_child(json)

html_code = ('<style>'
         '#title{'
             'position: fixed;'
             'top: 50px;'
             'left: 50px;'
             'width:33%;'
             'z-index:9999;'
             'background-color: #D3D3D3;'
             'border-radius: 25px;'
             'border: 3px solid #000000;'
             'opacity: 0.62;'
         '}'
         '.center {'
            'display: block;'
            'margin-left: auto;'
            'margin-right: auto;'

         '}'
     '</style>'     
     '<div id="title">'
        '<br>'     
        '<img src="https://debboutr.github.io/epa_small.png" alt="no_bueno" class="center">'
        '<a href="http://www.hexsim.net/" target="_blank"><h3 style="color:#000000; text-align: center;">Hex Sim</h3></a>'
        '<p style="text-align:center;">HexSim is a free, versatile, multi-species, life history simulator ideal for building models '
             'of animal and plant population viability, interactions, and responses to disturbance. HexSim '
             'models are spatially-explicit and individual-based, and HexSim individuals can be assigned '
             'dynamic life history traits. HexSim also includes a full genetics sub-model, making it a '
             'true eco-evo simulator. </p>'
     '</div>')

elem = folium.Element(html_code)
hex_map.get_root().html.add_child(elem)

hex_map.save("watermap.html")
hex_map